In [1]:
import tkinter as tk
from tkinter import ttk
from deriv_api import DerivAPI
import asyncio
import threading

### Account info 

In [2]:
# Store the account data globally
accounts_data = []
api = None
mt5_lst = []

def save_account_info(app_id_entry, api_key_entry, IP_key_entry=None):
    app_id = int(app_id_entry.get())
    api_key = api_key_entry.get()
    #IP_address = IP_key_entry.get()
    
    # Add the account data to the global list
    account = {"app_id": app_id, "api_key": api_key, "IP":None, "status":False}
    
    if account not in accounts_data:
        accounts_data.append(account)
        thread = threading.Thread(target=asyncio.run, args=(start_authorization(api_key, app_id),))
        thread.start()
    
    # Clear the entry fields after saving account info
    app_id_entry.delete(0, tk.END)
    api_key_entry.delete(0, tk.END)
    #IP_key_entry.delete(0, tk.END)

def show_info_page(root):
    root.destroy()  # Close the account information page
    show_info_window()

def show_account_info_page(previous_root=None):
    root = tk.Tk()
    root.title("Account Information")

    def save_and_continue():
        save_account_info(app_id_entry, api_key_entry)
        balance_button.config(state=tk.NORMAL)
        transfer_button.config(state=tk.NORMAL)
        continue_button.config(state=tk.NORMAL)  # Enable the "Continue" button after saving
    
    # Labels for input fields
    app_id_label = ttk.Label(root, text="App ID:")
    app_id_label.grid(row=0, column=0, padx=5, pady=5, sticky=tk.E)
    api_key_label = ttk.Label(root, text="API Key:")
    api_key_label.grid(row=1, column=0, padx=5, pady=5, sticky=tk.E)
    IP_address_label = ttk.Label(root, text="IP Address:")
    IP_address_label.grid(row=2, column=0, padx=5, pady=5, sticky=tk.E)

    # Entry widgets for user input
    app_id_entry = ttk.Entry(root)
    app_id_entry.grid(row=0, column=1, padx=5, pady=5)
    api_key_entry = ttk.Entry(root)
    api_key_entry.grid(row=1, column=1, padx=5, pady=5)
    IP_key_entry = ttk.Entry(root)
    IP_key_entry.grid(row=2, column=1, padx=5, pady=5)
    
    IP_key_entry.config(state=tk.DISABLED)

    # Save button
    save_button = ttk.Button(root, text="Save Account", command=save_and_continue)
    save_button.grid(row=3, column=0, columnspan=2, padx=5, pady=10)
    
    #view balance
    balance_button = ttk.Button(root, text="Balance", command=lambda: create_balance_page(root))
    balance_button.grid(row=5, column=0, columnspan=2, padx=5, pady=10)
   
    
    # Continue button to go to balance page
    continue_button = ttk.Button(root, text="View Information", command=lambda: show_info_page(root))
    continue_button.grid(row=4, column=0, columnspan=2, padx=5, pady=10)
    
    
    #transfer button
    transfer_button = ttk.Button(root, text="Fund Transfer", command=lambda: transaction_selection(root))
    transfer_button.grid(row=6, column=0, columnspan=2, padx=5, pady=10)
    

    # Close the previous window when navigating back
    if previous_root:
        previous_root.destroy()
    else:
        balance_button.config(state=tk.DISABLED)
        transfer_button.config(state=tk.DISABLED)
        continue_button.config(state=tk.DISABLED)  

    root.mainloop()

def show_info_window():
    info_root = tk.Tk()
    info_root.title("Balance Information")

    # Create column headers with modern style
    header_labels = ["App ID", "API Key", "IP address", "Status"]
    for i, header in enumerate(header_labels):
        header_label = ttk.Label(info_root, text=header, font=("Arial", 12, "bold"))
        header_label.grid(row=0, column=i, padx=5, pady=5, sticky=tk.W)

    # Populate the balance data dynamically
    for i, data in enumerate(accounts_data):
        app_id_label = ttk.Label(info_root, text=data["app_id"])
        app_id_label.grid(row=i+1, column=0, padx=5, pady=5, sticky=tk.W)

        api_key_label = ttk.Label(info_root, text=data["api_key"])
        api_key_label.grid(row=i+1, column=1, padx=5, pady=5, sticky=tk.W)
        
        if data["IP"] is not None:
            IP_address_label = ttk.Label(info_root, text=data["IP"])
            IP_address_label.grid(row=i+1, column=2, padx=5, pady=5, sticky=tk.W)
        
        status_label = ttk.Label(info_root, text=data["status"])
        status_label.grid(row=i+1, column=2, padx=5, pady=5, sticky=tk.W)
    
    back_button = ttk.Button(info_root, text="Back", command=lambda: show_account_info_page(info_root))
    back_button.grid(row=len(accounts_data)+1, column=0, padx=5, pady=10, columnspan=2)

    info_root.mainloop()

### AUTHORIZATION AND BALANCE FETCH

async def fetch_balance(wallet):
    response = await api.mt5_login_list()
    
    for i in response["mt5_login_list"]:
        mt5_data = {}
        mt5_data['email'] = i['email']
        mt5_data['loginid'] = i['login']
        mt5_data['balance'] = i['balance']
        mt5_data['wallet'] = wallet

        if mt5_data not in mt5_lst:
            mt5_lst.append(mt5_data)

    print(mt5_lst)
    
async def do_authorization(root, api_token, app_id):
    try:
        global api
        api = DerivAPI(app_id=app_id)
        authorize = await api.authorize(api_token)
        
        wallet = authorize["authorize"]['balance']
        
        print("Authorization successful")
        
        for i, acc_data in enumerate(accounts_data):
            if acc_data['api_key'] == api_token and acc_data['app_id'] == app_id:
                acc_data['status'] = True
        print(accounts_data)
 
        await fetch_balance( wallet)
       
    except Exception as e:
        print(f'Authorization error occurred: {e}')
    
    root.destroy()

async def start_authorization(api_token, app_id):
    root = tk.Tk()
    root.title('Authorizing')
    root.minsize(200 ,200)
    label = ttk.Label(root, text="Authorizing...Please wait...")
    label.pack(pady=10)

    thread1 = threading.Thread(target=asyncio.run, args=(do_authorization(root, api_token, app_id),))
    thread1.start()

    root.mainloop()
    
def create_balance_page(window):
    window.destroy()
    window = tk.Tk()
    window.title("Balance")

    columns = ["Account Email", "Mt5 Login ID", "Balance", "Wallet $$", "Total"]
    
    treeview = ttk.Treeview(window, columns=columns, show='headings')
    treeview.pack(padx = 10, pady = 10)
    
    #treeview.column('#0', width=10)
    for col_id, heading in enumerate(columns, start=1):
        treeview.heading(f'#{col_id}', text=heading)
        treeview.column(f'#{col_id}', width=150)
        
    style = ttk.Style()
    style.configure("Treeview", borderwidth=1, relief="solid")
    style.configure("Treeview.Heading", borderwidth=1, relief="solid")
    style.map("Treeview", background=[('selected', '#0078d7')])
        
    email_totals = {}
    
    for data in mt5_lst:
        if data['email'] in email_totals:
            email_totals[data['email']] += data['balance']
        else:
            email_totals[data['email']] = data['balance']
        
    
    # Insert data into the Treeview
    for data in mt5_lst:
        email = data['email']
        loginid = data['loginid']
        balance = data['balance']
        wallet = data['wallet']
        total = email_totals[email] + wallet
        treeview.insert("", "end", values=(email, loginid, balance, wallet, total))

    back_button = ttk.Button(window, text="Back", command=lambda: show_account_info_page(window))
    back_button.pack(padx=10, pady=10)
    # Add a "Total" row
    #total_row = ["Total", "", "", sum(row[3] for row in data), sum(row[4] for row in data)]
    #treeview.insert("", "end", values=total_row)
    
    window.mainloop()    
    
def transaction_selection(window):
    window.destroy()
    window = tk.Tk()
    window.title("Select Fund Transfer")
    window.minsize(400, 200)
    
    label = ttk.Label(window, text="Select an account to transfer funds from")
    label.pack(padx=20, pady=10)
    
    dropdown_var = tk.StringVar()
    
    email_list = []
    
    for data in mt5_lst:
        if data['email'] not in email_list:
            email_list.append(data['email'])
    
    dropdown = ttk.Combobox(window, textvariable=dropdown_var, values=email_list)
    dropdown.pack(padx=20, pady=10)
    
    def extract_selection():
        selected_item = dropdown_var.get()
        if selected_item:
            create_transaction_window(window)
            print(f"Selected item: {selected_item}")
        else:
            print("No item selected")
    
    select_button= ttk.Button(window, text="Select", command = extract_selection)
    select_button.pack(padx=20, pady=10)
    
    back_button = ttk.Button(window, text="Back", command=lambda: show_account_info_page(window))
    back_button.pack(padx=20, pady=10)
    
    window.mainloop()

    
# Show the account information page initially
show_account_info_page()


Authorization successful
[{'app_id': 37949, 'api_key': 'HkCo0RHU9wCvjIr', 'IP': None, 'status': True}]
[{'email': 'tnafisul@gmail.com', 'loginid': 'MTR41450086', 'balance': 1.2, 'wallet': 10000}, {'email': 'tnafisul@gmail.com', 'loginid': 'MTR41450088', 'balance': 0, 'wallet': 10000}, {'email': 'tnafisul@gmail.com', 'loginid': 'MTR41450089', 'balance': 0, 'wallet': 10000}, {'email': 'tnafisul@gmail.com', 'loginid': 'MTR80063975', 'balance': 0, 'wallet': 10000}]


### Transactions

In [6]:
def create_transaction_window(window, username):
    window.destroy()
    window = tk.Tk()
    window.title("Internal fund transfer")
    window.minsize(600, 200)
    
    radio_frame = ttk.Frame(window)
    radio_frame.pack(padx=10, pady=10)
    
    username_label = ttk.Label(radio_frame, text=username)
    username_label.grid(row=0, column=3, padx=20, pady=10, sticky=tk.W)

    for idx, data in enumerate(mt5_lst):
        if data['email'] == username:
            subaccounts_radio = ttk.Radiobutton(
                radio_frame,
                text=f"{data['loginid']}   {data['balance']}",
                value=data['loginid']
            )
            subaccounts_radio.grid(row=idx+1, column=1, padx=5, pady=5, sticky=tk.W)

    transfer_amount_label = ttk.Label(radio_frame, text="Enter amount to transfer: ")
    transfer_amount_label.grid(row=1, column=3, columnspan=3, padx=10, pady=5)
    transfer_entry = ttk.Entry(radio_frame)
    transfer_entry.grid(row=2, column=3, columnspan=3, padx=10, pady=5)
    transfer_button = ttk.Button(radio_frame, text="Transfer")
    transfer_button.grid(row=3, column=3, columnspan=2, padx=10, pady=5)

    # Create the dropdown widget for usernames
    receiver_label = ttk.Label(radio_frame, text="Receiver")
    receiver_label.grid(row=1, column=6, columnspan=3, padx=10, pady=5)
    
    mt5_account_list = []
    
    for data in mt5_lst:
        mt5_account_list.append(data['loginid'])
    
    dropdown_var = tk.StringVar()
    dropdown = ttk.Combobox(radio_frame, textvariable=dropdown_var, values=mt5_account_list)
    dropdown.grid(row=2, column=6, padx=10, pady= 5, sticky=tk.W)
    
    back_button = ttk.Button(radio_frame, text="Back", command=lambda: show_account_info_page(window))
    back_button.grid(row=6, column=3, columnspan=2, padx=10, pady=5)
    
    window.mainloop()
    
window = tk.Tk()
create_transaction_window(window, 'tnafisul@gmail.com')
